In [2]:
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
import re

## Dataset creation

In [3]:
# import os
# import librosa
# import soundfile as sf

# # Assuming 'output_filename' is the desired filename for the output WAV file
# output_filename = 'long_form_audio/concatenated_output.wav'
# data=pd.read_csv('../MADHAV_task_1/Task3/metadata_test_clean.tsv', sep='\t', header=None, names=['path', 'text', 'num','sr'])
# # List to store audio chunks
# audio_chunks = []
# target=""
# # Iterate over all files in the directory
# for i in range(10):
#     file_path = f"output/audio_noise{i}.wav"

#     # Load the audio file
#     audio, sr = librosa.load(file_path, sr=None)
#     if(target==""):
#         target=data['text'][i]
#     else:
#         target+=" "+data['text'][i]
#     # Append the audio to the list
#     audio_chunks.append(audio)

# # Concatenate all audio chunks
# concatenated_audio = np.concatenate(audio_chunks)

# # Save the concatenated audio to a new WAV file
# sf.write(output_filename, concatenated_audio, samplerate=sr)
# print(target)

In [4]:
# %cd /home/suryansh/MADHAV

In [5]:
# txt_file_path = "./hindi_data/givenFolderHSp/txt/12janA.txt"

# # Open the file with UTF-8 encoding
# with open(txt_file_path, 'r') as file:
#     target = file.read()

# # Now 'hindi_text' contains the contents of the file as a string
# print(target)

## Model algorithm

In [6]:
def split_audio_using_VAD(input_file, pred, chunk_length_sec=15):
    # Load the audio file
    audio, sr = librosa.load(input_file, sr=None)
    timestamps_df=pd.DataFrame()
    timestamps_df['start']=pred['start'].apply(lambda x: x*sr/1000)
    timestamps_df['end']=pred['end'].apply(lambda x: x*sr/1000)
    print(timestamps_df)
    # Calculate the total duration of the audio in samples
    total_samples = len(audio)

    # Calculate the chunk length in samples
    chunk_length_samples = int(sr * chunk_length_sec)
    print(chunk_length_samples)
    # Create an array to store audio chunks
    audio_chunks = []
    start_sample=0
    end_sample=chunk_length_samples
    split_sec=[]
    k=0
    index=0
    for i, row in timestamps_df.iterrows():
        if (start_sample+chunk_length_samples)>=row['end']:
            end_sample=row['end']
        else:
            audio_chunks.append(audio[int(start_sample):int(end_sample)])
            split_sec.append((index, int(start_sample)/sr, int(end_sample)/sr))
            index+=1
            start_sample=end_sample
            end_sample=row['end']
            while(start_sample+chunk_length_samples<row['end']):
                end_sample=start_sample+chunk_length_samples
                audio_chunks.append(audio[int(start_sample):int(end_sample)])
                split_sec.append((index, int(start_sample)/sr, int(end_sample)/sr))
                start_sample=end_sample
                index+=1
                end_sample=row['end']
            

    last_chunk = audio[int(start_sample):int(total_samples)]
    audio_chunks.append(last_chunk)
    split_sec.append((index, start_sample/sr, total_samples/sr))
    split_sec=pd.DataFrame(split_sec, columns=['index', 'start', 'end'])
    return audio_chunks, split_sec

In [7]:
import torch
torch.set_num_threads(1)

from IPython.display import Audio
from pprint import pprint
# download example
torch.hub.download_url_to_file('https://models.silero.ai/vad_models/en.wav', 'en_example.wav')

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)

(get_speech_timestamps,
 _, read_audio,
 *_) = utils
sampling_rate=16000

100%|██████████████████████████████████████| 1.83M/1.83M [00:01<00:00, 1.22MB/s]
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /home/suryansh/.cache/torch/hub/master.zip


In [8]:
# input_file="./hindi_data/givenFolderHSp/fullAudio/12janA.wav"
# wav = read_audio(input_file, sampling_rate=sampling_rate)
# # get speech timestamps from full audio file
# speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=sampling_rate)
# df = pd.DataFrame(speech_timestamps)
# df = df // 16
# df

In [9]:
# # Split the audio into 15-second chunks with adjustment for the last chunk
# chunks_array, split_sec = split_audio_using_VAD(input_file, df, 7)

# # Now, chunks_array contains the audio chunks as numpy arrays
# for i, chunk in enumerate(chunks_array):
#     print(f"Chunk {i}: start: {split_sec['start'][i]} end: {split_sec['end'][i]}")

In [10]:
# !pip install -q espnet_model_zoo

In [11]:
# #@title Choose English ASR model { run: "auto" }

# lang = 'en'
# fs = 16000 #@param {type:"integer"}
# tag = 'viks66/asr_train_asr_raw_hindi_bpe500' #@param ["Shinji Watanabe/spgispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_unnorm_bpe5000_valid.acc.ave", "kamo-naoyuki/librispeech_asr_train_asr_conformer6_n_fft512_hop_length256_raw_en_bpe5000_scheduler_confwarmup_steps40000_optim_conflr0.0025_sp_valid.acc.ave"] {type:"string"}

In [12]:
# !git lfs install
# !git clone https://huggingface.co/viks66/asr_train_asr_raw_hindi_bpe500

In [13]:
%cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500

/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500


In [14]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader()
# It may takes a while to download and build models
speech2text = Speech2Text(
    "exp/asr_train_asr_raw_hindi_bpe500/config.yaml",
    "exp/asr_train_asr_raw_hindi_bpe500/valid.acc.ave_10best.pth",
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.5,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

In [15]:
# import pandas as pd
# import librosa.display
# from IPython.display import display, Audio
# import matplotlib.pyplot as plt
# import time
# import numpy as np
# import soundfile as sf

# preds = ""

# for i, chunk in enumerate(chunks_array):
#     start_time = time.time()
#     speech = np.array([])
#     duration=split_sec['end'][i]*1000-split_sec['start'][i] * 1000
#     for _, row in df.iterrows():
#         start_sample = row['start']-split_sec['start'][i] * 1000
#         end_sample = row['end']-split_sec['start'][i] * 1000
# #         print(start_sample, end_sample, duration)
#         if(start_sample<0 and end_sample<0):
#             continue
#         if(start_sample>duration and end_sample>duration):
#             break
# #         print("Y")
#         speech = np.concatenate([speech, chunk[int(max(0,start_sample))*16:int(min(duration, end_sample))*16]])

#     print(len(speech))
#     nbests = speech2text(speech)
#     if len(speech) != 0:
# #         nbests = speech2text(speech)
#         text, *_ = nbests[0]
# #         output_filename = f'results/output_chunk{i}.wav'
# #         sf.write(output_filename, speech, samplerate=16000)
#         if(preds==""):
#             preds=(text_normalizer(text))
#         else:
#             preds += " " + (text_normalizer(text))

#     print(i, "/500")
#     elapsed_time = time.time() - start_time
#     print(f"Time taken: {elapsed_time:.2f} seconds")


In [16]:
import os
def fetch_filenames(directory_path, num_files=10):
    # Ensure the specified path is a directory
    if not os.path.isdir(directory_path):
        raise ValueError(f"The provided path '{directory_path}' is not a directory.")

    # Get all files in the directory
    all_files = os.listdir(directory_path)

    # Filter out only files (not directories)
    file_names = [file for file in all_files if os.path.isfile(os.path.join(directory_path, file))]

    # Take the first 'num_files' files (or all files if there are fewer than 'num_files')
    selected_files = file_names[:num_files]

    return selected_files


In [23]:
predictions=[]
ground_truth=[]

In [ ]:
import pandas as pd
import librosa.display
from IPython.display import display, Audio
import matplotlib.pyplot as plt
import time
import numpy as np
import soundfile as sf
from funasr_onnx import Fsmn_vad
%cd /home/suryansh/MADHAV
directory_path = "./hindi_data/givenFolderHSp/fullAudio"
num_files_to_fetch = 10
file_names = fetch_filenames(directory_path, num_files_to_fetch)
files = ["./hindi_data/givenFolderHSp/fullAudio/"+file_name for file_name in file_names]
txts=["./hindi_data/givenFolderHSp/txt/"+file_name.replace("wav", "txt") for file_name in file_names]
# files=['./hindi_data/givenFolderHSp/fullAudio/12janA.wav', './hindi_data/givenFolderHSp/fullAudio/11janA.wav','./hindi_data/givenFolderPBData/fullAudio/8483_FullAudio.wav','./hindi_data/givenPMS/newGivenPMS/longAudio/april_v.wav' ]
# txts=['./hindi_data/givenFolderHSp/txt/12janA.txt', './hindi_data/givenFolderHSp/txt/11janA.txt','./hindi_data/givenFolderPBData/txt/8483_FullAudio.txt','./hindi_data/givenPMS/newGivenPMS/txt/april_v.txt']
# files=['./hindi_data/givenFolderPBData/fullAudio/17162_FullAudio.wav']
# txts=['./hindi_data/givenFolderPBData/txt/17162_FullAudio.txt']
for txt_file_path, input_file in zip(txts, files):
    %cd /home/suryansh/MADHAV
    file=open(txt_file_path, 'r')
    target=""
    target=file.readlines()
    target=re.sub(" +", " ", " ".join(target))
    # Now 'hindi_text' contains the contents of the file as a string
#     print(target)
    ground_truth.append(target)
#     print(ground_truth)
    wav = read_audio(input_file, sampling_rate=sampling_rate)
    # get speech timestamps from full audio file
    speech_timestamps = get_speech_timestamps(wav, model, sampling_rate=sampling_rate, threshold=0.4)
    df = pd.DataFrame(speech_timestamps)
    df = df // 16
    # Split the audio into 15-second chunks with adjustment for the last chunk
    chunks_array, split_sec = split_audio_using_VAD(input_file, df, 7)
    %cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500
    preds = ""
    for i, chunk in enumerate(chunks_array):
        start_time = time.time()
        speech = np.array([])
        duration=split_sec['end'][i]*1000-split_sec['start'][i] * 1000
        for _, row in df.iterrows():
            start_sample = row['start']-split_sec['start'][i] * 1000
            end_sample = row['end']-split_sec['start'][i] * 1000
    #         print(start_sample, end_sample, duration)
            if(start_sample<0 and end_sample<0):
                continue
            if(start_sample>duration and end_sample>duration):
                break
    #         print("Y")
            speech = np.concatenate([speech, chunk[int(max(0,start_sample))*16:int(min(duration, end_sample))*16]])

        print(len(speech))
    #     nbests = speech2text(chunk)
        if len(speech) >= 10000:
            nbests = speech2text(speech)
            text, *_ = nbests[0]
    #         output_filename = f'results/output_chunk{i}.wav'
    #         sf.write(output_filename, speech, samplerate=16000)
            if(preds==""):
                preds=(text_normalizer(text))
            else:
                preds += " " + (text_normalizer(text))

        print(i, "/", len(chunks_array))
        elapsed_time = time.time() - start_time
        print(f"Time taken: {elapsed_time:.2f} seconds")
    predictions.append(preds)
    torch.cuda.empty_cache() 

/home/suryansh/MADHAV
/home/suryansh/MADHAV
          start         end
0       25120.0    104416.0
1      111136.0    144864.0
2      149024.0    182752.0
3      187424.0    220640.0
4      230944.0    265696.0
..          ...         ...
292  11444256.0  11457504.0
293  11460128.0  11482080.0
294  11485216.0  11495904.0
295  11499040.0  11528672.0
296  11540512.0  11568096.0

[297 rows x 2 columns]
112000
/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500
79296
0 / 132
Time taken: 3.34 seconds
67456
1 / 132
Time taken: 1.25 seconds
67968
2 / 132
Time taken: 1.25 seconds
89472
3 / 132
Time taken: 1.77 seconds
87872
4 / 132
Time taken: 1.72 seconds
99648


/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


5 / 132
Time taken: 1.67 seconds
52608
6 / 132
Time taken: 0.84 seconds
93120
7 / 132
Time taken: 1.62 seconds
99200
8 / 132
Time taken: 2.06 seconds
91392
9 / 132
Time taken: 1.07 seconds
55616
10 / 132
Time taken: 0.86 seconds
62912
11 / 132
Time taken: 1.24 seconds
79232
12 / 132
Time taken: 1.45 seconds
69440
13 / 132
Time taken: 1.18 seconds
83264
14 / 132
Time taken: 1.55 seconds
75584
15 / 132
Time taken: 1.39 seconds
53184
16 / 132
Time taken: 0.81 seconds
94016
17 / 132
Time taken: 1.59 seconds
45440
18 / 132
Time taken: 0.76 seconds
80768
19 / 132
Time taken: 1.37 seconds
50624
20 / 132
Time taken: 0.84 seconds
102336
21 / 132
Time taken: 1.68 seconds
67520
22 / 132
Time taken: 1.19 seconds
84288
23 / 132
Time taken: 1.56 seconds
86848
24 / 132
Time taken: 1.57 seconds
13248
25 / 132
Time taken: 0.20 seconds
88384
26 / 132
Time taken: 1.61 seconds
70656
27 / 132
Time taken: 1.24 seconds
39872
28 / 132
Time taken: 0.78 seconds
88448
29 / 132
Time taken: 1.69 seconds
74048
30 /

/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


0 / 116
Time taken: 0.91 seconds
0
1 / 116
Time taken: 0.00 seconds
62144
2 / 116
Time taken: 1.19 seconds
45312
3 / 116
Time taken: 0.85 seconds
64448
4 / 116
Time taken: 1.19 seconds
72576
5 / 116
Time taken: 1.41 seconds
95616
6 / 116
Time taken: 2.02 seconds
87872
7 / 116
Time taken: 1.64 seconds
99200
8 / 116
Time taken: 1.95 seconds
37824
9 / 116
Time taken: 0.77 seconds
13120
10 / 116
Time taken: 0.25 seconds
23168
11 / 116
Time taken: 0.42 seconds
77248
12 / 116
Time taken: 1.55 seconds
39360
13 / 116
Time taken: 0.74 seconds
105920
14 / 116
Time taken: 2.30 seconds
51648
15 / 116
Time taken: 0.99 seconds
93056
16 / 116
Time taken: 1.78 seconds
86976
17 / 116
Time taken: 1.78 seconds
93568
18 / 116
Time taken: 1.89 seconds
97152
19 / 116
Time taken: 1.91 seconds
92992
20 / 116
Time taken: 1.86 seconds
99840
21 / 116
Time taken: 1.98 seconds
37312
22 / 116
Time taken: 0.63 seconds
64960
23 / 116
Time taken: 1.22 seconds
107840
24 / 116
Time taken: 2.29 seconds
96832
25 / 116
Tim

...

8 / 131
Time taken: 2.34 seconds
75328
9 / 131
Time taken: 1.40 seconds
58816
10 / 131
Time taken: 0.91 seconds
86464
11 / 131
Time taken: 1.55 seconds
74176
12 / 131
Time taken: 1.42 seconds
98112
13 / 131
Time taken: 2.11 seconds
102720
14 / 131
Time taken: 1.80 seconds
85632
15 / 131
Time taken: 1.47 seconds
38336
16 / 131
Time taken: 0.26 seconds
7104
17 / 131
Time taken: 0.00 seconds
46464
18 / 131
Time taken: 0.81 seconds
0
19 / 131
Time taken: 0.00 seconds
37312
20 / 131
Time taken: 0.69 seconds
68992
21 / 131
Time taken: 1.52 seconds
76608
22 / 131
Time taken: 1.72 seconds
50624
23 / 131
Time taken: 0.97 seconds
0
24 / 131
Time taken: 0.00 seconds
0
25 / 131
Time taken: 0.00 seconds
0
26 / 131
Time taken: 0.00 seconds
25024
27 / 131
Time taken: 0.48 seconds
103808
28 / 131
Time taken: 2.21 seconds
94656
29 / 131
Time taken: 1.82 seconds
89408
30 / 131
Time taken: 1.93 seconds
91520
31 / 131
Time taken: 1.16 seconds
85312
32 / 131
Time taken: 1.81 seconds
76608
33 / 131
Time tak

/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


32 / 148
Time taken: 1.77 seconds
103360
33 / 148
Time taken: 2.07 seconds
42944
34 / 148
Time taken: 0.93 seconds
91584
35 / 148
Time taken: 1.94 seconds
22464
36 / 148
Time taken: 0.36 seconds
104256
37 / 148
Time taken: 2.27 seconds
88640
38 / 148
Time taken: 1.78 seconds
41408
39 / 148
Time taken: 0.85 seconds
105856
40 / 148
Time taken: 2.35 seconds
72576
41 / 148
Time taken: 1.34 seconds
59328
42 / 148
Time taken: 1.15 seconds
80320
43 / 148
Time taken: 1.63 seconds
36800
44 / 148
Time taken: 0.72 seconds
97216
45 / 148
Time taken: 1.99 seconds
54720
46 / 148
Time taken: 1.10 seconds
82304
47 / 148
Time taken: 1.82 seconds
83904
48 / 148
Time taken: 1.67 seconds
76608
49 / 148
Time taken: 1.43 seconds
56960
50 / 148
Time taken: 1.07 seconds
97664
51 / 148
Time taken: 1.79 seconds
34752
52 / 148
Time taken: 0.65 seconds
80832
53 / 148
Time taken: 1.42 seconds
71104
54 / 148
Time taken: 1.48 seconds
42432
55 / 148
Time taken: 0.76 seconds
102208
56 / 148
Time taken: 2.20 seconds
93

/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


0 / 131
Time taken: 0.95 seconds
0
1 / 131
Time taken: 0.00 seconds
66496
2 / 131
Time taken: 1.24 seconds
61376
3 / 131
Time taken: 1.23 seconds
50112
4 / 131
Time taken: 0.92 seconds
85376
5 / 131
Time taken: 1.60 seconds
41408
6 / 131
Time taken: 0.76 seconds
105408
7 / 131
Time taken: 2.14 seconds
71104
8 / 131
Time taken: 1.25 seconds
60864
9 / 131
Time taken: 0.75 seconds
64960
10 / 131
Time taken: 1.32 seconds
79808
11 / 131
Time taken: 1.61 seconds
81280
12 / 131
Time taken: 1.60 seconds
37696
13 / 131
Time taken: 0.70 seconds
77888
14 / 131
Time taken: 1.46 seconds
95104
15 / 131
Time taken: 2.00 seconds
82816
16 / 131
Time taken: 1.68 seconds
73088
17 / 131
Time taken: 1.48 seconds
90944
18 / 131
Time taken: 1.95 seconds
101696
19 / 131
Time taken: 2.28 seconds
50112
20 / 131
Time taken: 0.94 seconds
80320
21 / 131
Time taken: 1.59 seconds
58176
22 / 131
Time taken: 1.27 seconds
74688
23 / 131
Time taken: 1.64 seconds
69552
24 / 131
Time taken: 1.50 seconds
74688
25 / 131
Tim

In [ ]:
print(ground_truth)

In [ ]:
print(predictions)

In [ ]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(predictions, ground_truth))
wer = WordErrorRate()
print("WER:", wer(predictions, ground_truth))

In [ ]:
print("CER")
for i in range(10):
    cer = CharErrorRate()
    print(float(cer(predictions[i], ground_truth[i])))
print("WER")
for i in range(10):
    wer = WordErrorRate()
    print(float(wer(predictions[i], ground_truth[i])))

In [36]:
[txt.split('/')[-1] for txt in txts]

['27_feb_23.txt',
 '10janA.txt',
 '17janA.txt',
 '12janA.txt',
 '2023_04_14.txt',
 '2023_04_16.txt',
 '14janA.txt',
 '2023_04_09.txt',
 '28_feb_23.txt',
 '17_mar_noon.txt']

In [23]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(predictions[:-1], ground_truth[:-1]))
wer = WordErrorRate()
print("WER:", wer(predictions[:-1], ground_truth[:-1]))

CER: tensor(0.1440)
WER: tensor(0.3162)


In [24]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(predictions[2], ground_truth[2]))
wer = WordErrorRate()
print("WER:", wer(predictions[2], ground_truth[2]))

CER: tensor(0.1480)
WER: tensor(0.3167)


In [25]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(predictions[3], ground_truth[3]))
wer = WordErrorRate()
print("WER:", wer(predictions[3], ground_truth[3]))

CER: tensor(0.7892)
WER: tensor(0.9279)


In [82]:
%cd /home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500

/home/suryansh/MADHAV/asr_train_asr_raw_hindi_bpe500


In [83]:
import time
import torch
import string
from espnet_model_zoo.downloader import ModelDownloader
from espnet2.bin.asr_inference import Speech2Text


d = ModelDownloader()
# It may takes a while to download and build models
speech2text = Speech2Text(
    "exp/asr_train_asr_raw_hindi_bpe500/config.yaml",
    "exp/asr_train_asr_raw_hindi_bpe500/valid.acc.ave_10best.pth",
    device="cuda",
    minlenratio=0.0,
    maxlenratio=0.5,
    ctc_weight=0.3,
    beam_size=10,
    batch_size=0,
    nbest=1
)

def text_normalizer(text):
    text = text.upper()
    return text.translate(str.maketrans('', '', string.punctuation))

In [84]:
import pandas as pd
import soundfile
import librosa.display
from IPython.display import display, Audio
import matplotlib.pyplot as plt
import time

col_names=['path', 'text', 'time', 'sr', 'std']
egs = pd.read_csv("/home/suryansh/MADHAV/hindi_data/givenFolderHSp/tsv/12janA.tsv", sep="\t", names=col_names, header=None, encoding='utf-8', index_col=False)
# print(egs['path'])
preds=[]
target=[]
for index, row in egs.iterrows():
    start_time = time.time()
    speech, rate = soundfile.read("/home/suryansh/MADHAV/hindi_data/givenFolderHSp/finalSplitAudio/" + row["path"])
#     assert fs == int(row["sr"])
    nbests = speech2text(speech)

    text, *_ = nbests[0]
# Uncomment to see details of each prediction
#     print(f"Input Speech: ESPNet_asr_egs/{row['path']}")
    # let us listen to samples
#     display(Audio(speech, rate=rate))
#     librosa.display.waveshow(speech, sr=rate, color="blue")
#     plt.show()
#     print(f"Reference text: {text_normalizer(row['text'])}")
    target.append(text_normalizer(row['text']))
#     print(f"ASR hypothesis: {text_normalizer(text)}")
    preds.append(text_normalizer(text))
    print(index,"/500")
    elapsed_time = time.time() - start_time
    print(f"Time taken: {elapsed_time:.2f} seconds")

/home/suryansh/anaconda3/lib/python3.11/site-packages/espnet/nets/ctc_prefix_score.py:323: RuntimeWarning: invalid value encountered in logaddexp
  r_sum = self.xp.logaddexp(


0 /500
Time taken: 5.14 seconds
1 /500
Time taken: 5.37 seconds
2 /500
Time taken: 4.41 seconds
3 /500
Time taken: 2.48 seconds
4 /500
Time taken: 4.52 seconds
5 /500
Time taken: 4.49 seconds
6 /500
Time taken: 5.41 seconds
7 /500
Time taken: 5.35 seconds
8 /500
Time taken: 5.75 seconds
9 /500
Time taken: 5.21 seconds
10 /500
Time taken: 4.13 seconds
11 /500
Time taken: 5.15 seconds
12 /500
Time taken: 6.05 seconds
13 /500
Time taken: 5.67 seconds
14 /500
Time taken: 6.45 seconds
15 /500
Time taken: 5.18 seconds
16 /500
Time taken: 5.57 seconds
17 /500
Time taken: 5.97 seconds
18 /500
Time taken: 2.56 seconds
19 /500
Time taken: 5.08 seconds
20 /500
Time taken: 5.53 seconds
21 /500
Time taken: 2.00 seconds
22 /500
Time taken: 4.25 seconds
23 /500
Time taken: 5.51 seconds
24 /500
Time taken: 6.48 seconds
25 /500
Time taken: 5.60 seconds
26 /500
Time taken: 4.99 seconds
27 /500
Time taken: 5.11 seconds
28 /500
Time taken: 5.03 seconds
29 /500
Time taken: 5.10 seconds
30 /500
Time taken: 

In [85]:
from torchmetrics.text import CharErrorRate, WordErrorRate
cer = CharErrorRate()
print("CER:", cer(preds, target))
wer = WordErrorRate()
print("WER:", wer(preds, target))

CER: tensor(0.1677)
WER: tensor(0.3445)
